In [3]:
from biotoolsJSONParser1 import *
import json
import os
from utils import *

下载BioTools.json数据

In [3]:
path='D:/wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn/_school/source/jupyter/review/dataset/'
url='https://bio.tools/api/t/?format=json&'
downloadBiotoolsJson(path)

提取各个biotoolsID工具的publication下的doi pmid pmcid 和metadata下的abstract

In [8]:
path='D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\biotoolsjson'
toolLists=[]
Biotools2DOI=dict()
Biotools2PMID=dict()
Biotools2PMCID=dict()
abstractInfo=dict()
pmidInfo=dict() # 用于存放BioTools的url上发表过的，有pmid但是没有abtract的pmid,后面通过entraz查找
doiInfo=dict() # 用于存放BioTools的url上发表过的，没有abtract没有pmid只能通过doi查找的，后面通过web of science查找
nullPublicationCount=0 # 没有发表publication的biotoolsId数量
start_page=1
max_page=2841
# max_page=getMaxPage()
for p in range(start_page,max_page+1): 
    filename=path+'\\BioToolsJSON'+str(p)+'.json'
    datas=readJSON(filename)
    data=datas['list']
    for d in data:
        biotoolsId=getSimpleInfo(d,'biotoolsID')
        pub=getSimpleInfo(d,'publication')
        if(len(pub)>0):
            Biotools2DOI[biotoolsId]=pub[0]['doi']
            Biotools2PMID[biotoolsId]=pub[0]['pmid']
            Biotools2PMCID[biotoolsId]=pub[0]['pmcid']
            if(pub[0]['metadata'] is not None):
                if pub[0]['metadata']['abstract'] is not None :
                    abstract=pub[0]['metadata']['abstract']
                    abstractInfo[biotoolsId]=abstract
                else:
                    if pub[0]['pmid'] is not None :
                        pmidInfo[biotoolsId]=pub[0]['pmid']
                    else:
                        doiInfo[biotoolsId]=pub[0]['doi']
            else:
                if pub[0]['pmid'] is not None :
                    pmidInfo[biotoolsId]=pub[0]['pmid']
                else:
                    doiInfo[biotoolsId]=pub[0]['doi']
        else:
            nullPublicationCount=nullPublicationCount+1

print("没有关联论文的工具数量：",nullPublicationCount)
print("有关联论文，但是没有摘要的pmid数量：",len(pmidInfo))
print("有关联论文，但是没有摘要和pmid的doi数量：",len(doiInfo))
saveNpy('D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\output\\Biotools2DOI.npy',Biotools2DOI)
saveNpy('D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\output\\Biotools2PMID.npy',Biotools2PMID)
saveNpy('D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\output\\Biotools2PMCID.npy',Biotools2PMCID)
saveNpy('D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\output\\biotolsID2abstractDict.npy',abstractInfo)
saveNpy('D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\output\\biotolsID2pmidDict.npy',pmidInfo)
saveNpy('D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\output\\biotolsID2doiDict.npy',doiInfo)

没有关联论文的工具数量： 2137
有关联论文，但是没有摘要的pmid数量： 1507
有关联论文，但是没有摘要和pmid的doi数量： 3751


In [10]:
!pip install Bio

Defaulting to user installation because normal site-packages is not writeable


根据文献pmid去pubmed获取摘要，通过Entrez获取。数据在biotolsID2pmidDict.npy

In [11]:
from Bio import Entrez
from utils import loadNpy
import re
from operator import length_hint

Entrez.email = '1481995200@qq.com'
pmidInfo=loadNpy('D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\output\\biotolsID2pmidDict.npy').item()
abstractInfo=dict()
NullFindID=[]
# print(pmidInfo) <biotoolsId,pmid>
for id in pmidInfo:
    pmid=pmidInfo[id]
    try:
        handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract")
        record = Entrez.read(handle)
        # print(record)
        if record['PubmedArticle']:
            if 'Abstract' in record['PubmedArticle'][0]['MedlineCitation']['Article']:
                abstract_list=record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText']
                abstract=abstract_list[0].__repr__()
                length=length_hint(abstract_list)
                if length>1:
                    abstract='; '.join(abstract_list)
            else:
                abstract=' '
        else:
            abstract=' '
        abstractInfo[id]=abstract
        # print(abstract)
        handle.close()
    except Exception as e:
        print(id)
        NullFindID.append(id)
        
#要将新获得的论文摘要存入之前的文件吗？
print(len(pmidInfo))
saveNpy('D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\output\\biotolsID2abstractDict1.npy',abstractInfo)

其余的论文摘要通过doi在谷歌学术上获取，数据在biotolsID2doiDict.npy，在DownLoadGoogleScholarAbstractByDOI.py运行,未实现代理ip需隔段时间手动修改爬取范围

In [16]:
from DownLoadGoogleScholarAbstractByDOI import *
InPutPath = r'D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\biotolsID2doiDict.xlsx'
OutputPath = r'D:\\wenjiannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn\\_school\\dataset\\abstract.xlsx'
TableColumn = 2
for Tablerow in range(1,10):
    doi = readExcel(InPutPath, Tablerow, TableColumn)
    get_paper_list_by_doi(doi, Tablerow, TableColumn, max_capacity=100, debug_mode=False, save_file=OutputPath)
print("end")

https://scholar.google.com/scholar?hl=zh-CN&as_sdt=0%2C5&q=10.1101%2F2023.08.28.555103&btnG=
['N', 'o', 'n', '-', 'c', 'o', 'd', 'i', 'n', 'g', ' ', 'R', 'N', 'A', 's', ' ', 'p', 'l', 'a', 'y', ' ', 'a', ' ', 'g', 'r', 'e', 'a', 't', ' ', 'v', 'a', 'r', 'i', 'e', 't', 'y', ' ', 'o', 'f', ' ', 'r', 'o', 'l', 'e', 's', ' ', 'i', 'n', ' ', 'm', 'a', 'n', 'y', ' ', 'c', 'e', 'l', 'l', 'u', 'l', 'a', 'r', ' ', 'p', 'r', 'o', 'c', 'e', 's', 's', 'e', 's', ' ', 'w', 'i', 't', 'h', ' ', 't', 'h', 'e', 'i', 'r', ' ', 's', 'p', 'a', 't', 'i', 'a', 'l', ' ', 's', 't', 'r', 'u', 'c', 't', 'u', 'r', 'e', ' ', 'k', 'n', 'o', 'w', 'n', ' ', 't', 'o', ' ', 'd', 'i', 'c', 't', 'a', 't', 'e', ' ', 't', 'h', 'e', ' ', 'f', 'u', 'n', 'c', 't', 'i', 'o', 'n', 'i', 'n', 'g', '.', ' ', 'R', 'N', 'A', ' ', 's', 'e', 'c', 'o', 'n', 'd', 'a', 'r', 'y', ' ', 's', 't', 'r', 'u', 'c', 't', 'u', 'r', 'e', ' ', 'l', 'a', 'r', 'g', 'e', 'l', 'y', ' ', 'd', 'e', 't', 'e', 'r', 'm', 'i', 'n', 'e', 's', ' ', 't', 'h', '

KeyboardInterrupt: 